In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score

In [120]:
data = pd.read_csv('../../src/data/syriatel_customer_churn.csv')

In [121]:
data

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [123]:
# checking target balance
data.churn.value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [124]:
# Checking phone number
data['phone number'].value_counts()

# dropping this

360-3126    1
394-8086    1
416-1845    1
361-9839    1
344-5973    1
           ..
400-4344    1
334-4354    1
364-3567    1
337-3868    1
335-4584    1
Name: phone number, Length: 3333, dtype: int64

In [125]:
# Dropping phone number column, and assigning modified dataframe to new variable: df. To preserve og dataframe

df = data.drop('phone number', axis=1)

In [126]:
# checking df.info() to confirm drop

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   international plan      3333 non-null   object 
 4   voice mail plan         3333 non-null   object 
 5   number vmail messages   3333 non-null   int64  
 6   total day minutes       3333 non-null   float64
 7   total day calls         3333 non-null   int64  
 8   total day charge        3333 non-null   float64
 9   total eve minutes       3333 non-null   float64
 10  total eve calls         3333 non-null   int64  
 11  total eve charge        3333 non-null   float64
 12  total night minutes     3333 non-null   float64
 13  total night calls       3333 non-null   int64  
 14  total night charge      3333 non-null   

In [127]:
# checking area code value counts
df['area code'].value_counts()

# will keep this.

415    1655
510     840
408     838
Name: area code, dtype: int64

In [128]:
# splitting feature variables and target variable
X = df.drop('churn', axis=1)
y = df['churn']

In [129]:
# Checking dtypes of our feature variables
X.dtypes.value_counts()

float64    8
int64      8
object     3
dtype: int64

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [131]:
# checking shape of data sets
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2499, 19) (2499,)
(834, 19) (834,)


### Checking for duplicates

In [132]:
# checking for duplicates

X_train.duplicated().value_counts()

False    2499
dtype: int64

### Preprocessing

### Train data

In [133]:
# Splitting data into numerical and categoricals columns

X_tr_num = X_train.select_dtypes(exclude='object') # numerical cols
X_tr_cat = X_train.select_dtypes(include='object') # categorical cols

In [140]:
# ohe'ing train categorical cols 

ohe = OneHotEncoder(sparse= False, drop= 'first')

X_tr_ohe = pd.DataFrame(ohe.fit_transform(X_tr_cat), 
                        index= X_tr_cat.index, 
                        columns=ohe.get_feature_names(X_tr_cat.columns))
print(X_tr_ohe.shape)


# standardizing train numerical cols

ss = StandardScaler()

X_tr_sc = pd.DataFrame(ss.fit_transform(X_tr_num), 
                       index= X_tr_num.index, 
                       columns= X_tr_num.columns)
print(X_tr_sc.shape)

(2499, 52)
(2499, 16)


In [139]:
# joining the num and ohe cat cols together

X_tr_final = X_tr_sc.join(X_tr_ohe)
X_tr_final.head()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
367,-1.404508,-0.512381,-0.584700,-1.883677,1.330852,-1.884170,1.037727,0.401340,1.037905,1.069609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3103,0.366388,-0.512381,-0.584700,0.294083,0.529165,0.293703,0.516178,0.401340,0.517286,2.214376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549,0.518179,-0.679077,1.685101,1.056392,-1.875896,1.056666,0.093407,0.849774,0.094283,-0.077125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2531,2.010792,-0.512381,-0.584700,-0.679156,1.681590,-0.679320,-0.402459,0.650470,-0.403094,-0.322994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2378,0.290493,1.749923,-0.584700,0.484660,1.080325,0.484172,-0.718549,-0.296224,-0.719184,-1.186487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preproessing Test Data

In [144]:
# Splitting data into numerical and categoricals columns

X_te_num = X_test.select_dtypes(exclude='object') # numerical cols
X_te_cat = X_test.select_dtypes(include='object') # categorical cols

In [146]:
# ohe'ing test categorical cols 

X_te_ohe = pd.DataFrame(ohe.transform(X_te_cat), 
                        index= X_te_cat.index, 
                        columns=ohe.get_feature_names(X_te_cat.columns))
print(X_te_ohe.shape)


# standardizing test numerical cols

X_te_sc = pd.DataFrame(ss.fit_transform(X_te_num), 
                       index= X_te_num.index, 
                       columns= X_te_num.columns)
print(X_te_sc.shape)

(834, 52)
(834, 16)


In [147]:
# joining the num and ohe cat cols together

X_te_final = X_te_sc.join(X_te_ohe)
X_te_final.head()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
438,0.253426,1.631816,-0.613002,-0.432595,-0.364240,-0.432668,2.536832,0.276496,2.536592,-0.245813,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2674,-0.878934,-0.557108,-0.613002,-1.257063,0.812997,-1.256822,0.315785,1.201764,0.315986,-0.265922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1345,-0.115822,-0.557108,-0.613002,-3.216745,-4.926032,-3.216830,-0.818283,1.510186,-0.817401,-0.694241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1957,1.090388,-0.718397,-0.613002,0.605623,-1.050961,0.605977,0.054832,-0.494561,0.055145,-0.913427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,-0.165055,-0.718397,-0.613002,-0.630180,0.077224,-0.630254,0.459015,-1.419829,0.459102,0.524357,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## FSM

In [143]:
# instantiating logistic regression instance and fitting train data to model

lg1 = LogisticRegression()

lg1.fit(X_tr_final, y_train)

y_pred_lg1 = lg1.predict(X_test)

f1_score(y_test, y_pred_lg1)

ValueError: could not convert string to float: 'WY'